In [11]:
import cv2 #opencv库，用于读取图片等操作
import os
import numpy as np 
import pandas as pd  
from sklearn.preprocessing import StandardScaler #标准差标准化
from sklearn.svm import SVC                      #svm包中SVC用于分类
from sklearn.decomposition import PCA    

In [12]:
def get_data(x, y):
    file_path='./faces_4' #设置文件路径（这里为当前目录下的ORL文件夹）
    targets = os.listdir(file_path)
    train_set = np.zeros(shape=[1,960])  #train_set用于获取的数据集
    train_set = pd.DataFrame(train_set)     #将train_set转换成DataFrame类型
    target=[]  
    i = 0
    end = y
    for each in targets:
        cur_path = file_path + '/' + each + '/'
        i = i + 1
        img_names = os.listdir(cur_path)
        if y == 32 :
            end = len(img_names)
        for j in range(x, end):                # 分割图片
            target.append(i)    
            img_name = img_names[j]            # 图片名
            img = cv2.imread(cur_path + img_name, cv2.IMREAD_GRAYSCALE) #读取图片，第二个参数表示以灰度图像读入
            img = img.reshape(1,img.shape[0]*img.shape[1]) #将读入的图片数据转换成一维
            img = pd.DataFrame(img)           #将一维的图片数据转成DataFrame类型
            train_set = pd.concat([train_set,img],axis=0)#按行拼接DataFrame矩阵
    train_set.index = list(range(0,train_set.shape[0])) #设置 train_set的行索引
    train_set.drop(labels=0,axis=0,inplace=True) #删除初始化为0的行
    target = pd.DataFrame(target)             #将标签列表转成DataFrame类型
    return train_set,target  

In [13]:
'''读取数据集并划分训练集和测试集'''
x_train, y_train = get_data(0, 24) # 取前23张为训练集
x_test, y_test = get_data(24, 32)  # 取后几张为测试集

x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,950,951,952,953,954,955,956,957,958,959
1,9.0,34.0,20.0,17.0,56.0,41.0,50.0,49.0,45.0,43.0,...,106.0,108.0,76.0,99.0,119.0,73.0,93.0,63.0,23.0,3.0
2,8.0,33.0,20.0,16.0,55.0,40.0,49.0,48.0,44.0,42.0,...,98.0,81.0,86.0,119.0,94.0,75.0,73.0,54.0,13.0,2.0
3,8.0,31.0,19.0,15.0,51.0,38.0,46.0,45.0,42.0,40.0,...,44.0,72.0,92.0,63.0,81.0,106.0,56.0,58.0,17.0,1.0
4,9.0,34.0,20.0,16.0,55.0,40.0,49.0,48.0,45.0,43.0,...,19.0,47.0,79.0,77.0,86.0,100.0,82.0,62.0,76.0,56.0
5,45.0,19.0,25.0,58.0,45.0,52.0,45.0,44.0,42.0,40.0,...,99.0,69.0,94.0,69.0,56.0,59.0,22.0,9.0,1.0,0.0


In [16]:
'''数据标准化 标准差标准化'''
stdScaler = StandardScaler().fit(x_train) 
x_train = stdScaler.transform(x_train)
x_test = stdScaler.transform(x_test)

x_train[0:5]

array([[-1.31812529,  0.22018458, -1.50058831, ...,  0.41484541,
        -0.23778072, -0.66339101],
       [-1.38643669,  0.14320818, -1.50058831, ...,  0.19980675,
        -0.65708545, -0.72155132],
       [-1.38643669, -0.01074462, -1.61092569, ...,  0.29537949,
        -0.48936356, -0.77971162],
       [-1.31812529,  0.22018458, -1.50058831, ...,  0.39095222,
         1.98453432,  2.4191053 ],
       [ 1.14108493, -0.93446145, -0.94890143, ..., -0.87538653,
        -1.16025112, -0.83787193]])

In [ ]:
'''PCA降维'''
pca = PCA(n_components=20).fit(x_train) 
x_train = pca.transform(x_train) 
x_test = pca.transform(x_test)

x_train[0:5]

In [ ]:
'''建立SVM模型  默认为径向基核函数kernel='rbf' 多项式核函数kernel='poly' '''
svm = SVC(kernel='poly').fit(x_train, y_train)
svm.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9791666666666666

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
y_pred = svm.predict(x_test)
# 准确率
print("准确率：" + str(accuracy_score(y_test, y_pred)))

准确率：0.9791666666666666


In [ ]:
y_pred

array([ 1,  1,  1,  1, 12,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9, 10, 10, 10,
       10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 13,
       13, 13, 13, 13, 13, 13, 13,  8,  8, 14, 14, 14, 14, 14, 14, 15, 15,
       15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
       17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19,
       20, 20, 20, 20, 20, 20, 20, 20])